In [2]:
import qiskit
import covalent as ct

In [12]:
from qiskit.providers.ibmq import IBMQ

In [23]:
IBMQ.save_account('YOUR_API_KEY')

configrc.store_credentials:WARNING:2023-01-29 00:18:47,662: Credentials already present. Set overwrite=True to overwrite.


In [13]:
# This step 
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [34]:
from qiskit import QuantumCircuit, execute, transpile
import numpy as np

# @ct.electron
def construct_circuit():    
    circuit = QuantumCircuit(7)
    
    unitary_operators = [[[0,1],[1,0]]]
    
    for operator in unitary_operators:
        circuit.unitary(operator, 0)
    
    circuit.measure_all()
    
    return circuit

# @ct.electron
def construct_simulator(circuit):
    from qiskit.providers.ibmq import IBMQ
 
    provider = IBMQ.get_provider(hub='ibm-q-community', group='mit-hackathon', project='main')
    provider.backends()

    backend = provider.get_backend('ibm_oslo')
    
    from qiskit.providers.aer import AerSimulator

    # Generate a simulator the mimics the real quantum system with the latest calibration
    simulator = AerSimulator.from_backend(backend)
    circuit = transpile(circuit, simulator)
    
    return circuit, simulator    

# @ct.electron
def invoke_simulation(circuit, simulator):
    result = simulator.run(circuit).result()
    counts = result.get_counts(circuit)
    
    return circuit, result, counts
    
# @ct.lattice
def test():
    circuit = construct_circuit()
    circuit, simulator = construct_simulator(circuit=circuit)
    
    return invoke_simulation(circuit=circuit, simulator=simulator)

In [35]:
circuit, results, counts = test()

In [36]:
circuit.draw()

┌───┐ ░ ┌─┐                  
q_0 -> 0 ┤ X ├─░─┤M├──────────────────
         └───┘ ░ └╥┘┌─┐               
q_1 -> 1 ──────░──╫─┤M├───────────────
               ░  ║ └╥┘┌─┐            
q_2 -> 2 ──────░──╫──╫─┤M├────────────
               ░  ║  ║ └╥┘┌─┐         
q_3 -> 3 ──────░──╫──╫──╫─┤M├─────────
               ░  ║  ║  ║ └╥┘┌─┐      
q_4 -> 4 ──────░──╫──╫──╫──╫─┤M├──────
               ░  ║  ║  ║  ║ └╥┘┌─┐   
q_5 -> 5 ──────░──╫──╫──╫──╫──╫─┤M├───
               ░  ║  ║  ║  ║  ║ └╥┘┌─┐
q_6 -> 6 ──────░──╫──╫──╫──╫──╫──╫─┤M├
               ░  ║  ║  ║  ║  ║  ║ └╥┘
 meas: 7/═════════╩══╩══╩══╩══╩══╩══╩═
                  0  1  2  3  4  5  6